# Simple Demand Forecasting

This notebook was inspired by the "Keeping it Simple" Notebook by XYZT for the Kaggle competition on demand forecasting. https://www.kaggle.com/thexyzt/keeping-it-simple-by-xyzt

In [200]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import datetime

In [176]:
input_path = '../data/'
data_train = pd.read_csv(os.path.join(input_path, 'train.csv'),  parse_dates=['date'], index_col=['date'])
data_test = pd.read_csv(os.path.join(input_path, 'test.csv'), parse_dates=['date'], index_col=['date'])

## Make Features

In [205]:
def get_date_related_features(df):
    """
    Returns a DataFrame with date related features from df.
    Index column of input df needs to be a timestamp index in the format *'YYYY-MM-DD'*
    Calendar information include date, time, dayofweek, dayofmonth, dayofyear, weekofyear, month, year. 
    
    Args:
        df (DataFrame):  With a timestamp index in the format *'YYYY-MM-DD'*

    Returns:
        df: DataFrame with date related columns
    """
    df = (df
          .assign(date=df.index.date)
          .assign(dayofweek=df.index.dayofweek)
          .assign(dayofmonth=df.index.day)
          .assign(dayofyear=df.index.dayofyear)
          .assign(weekofyear=df.index.weekofyear)
          .assign(month=df.index.month)
          .assign(year=df.index.year)
          )
    return df


def output_csv(df_test, pred_col='pred_year'):
    today = datetime.datetime.today().strftime('%Y-%m-%d')
    (df_test[['id', pred_col]]
        .rename(columns={pred_col: 'sales'})
        .to_csv("submission_{}.csv".format(today), index=False)
    )
    return 

In [177]:
df_train = get_date_related_features(data_train)
df_test = get_date_related_features(data_test)

## Make Naive Predictors

In [181]:
def naive_predictor(df_train, df_test):
    groupby_columns = ['store', 'item', 'month', 'dayofmonth']
    
    df_naive = (df_train.groupby(groupby_columns)
                 .agg({'sales': 'mean'})
                 .rename(columns={'sales': 'pred'})
                 .reset_index()
               )
    
    df_train = df_train.merge(df_naive, 
                              left_on=groupby_columns, 
                              right_on=groupby_columns, 
                              how="left")
    
    df_test = df_test.merge(df_naive, 
                          left_on=groupby_columns, 
                          right_on=groupby_columns, 
                          how="left")

    df_train = df_train.sort_values(by=['store','item', 'date']).round()
    df_test = df_test.sort_values(by=['store','item', 'date']).round()

    return df_train, df_test

In [188]:
df_train_1, df_test_1 = naive_predictor(df_train, df_test)

## Naive Predictor 2

In [197]:
def naive_predictor_2(df_train, df_test):
    ## Find baseline estimate
    df_train, df_test = make_baseline(df_train, df_test)
    ## Find day of week and month scaling factors 
    df_train, df_test = make_calendar_adjustments(df_train, df_test)
    
    df_train = df_train.sort_values(by=['store','item', 'date']).round()
    df_test = df_test.sort_values(by=['store','item', 'date']).round()
    return df_train, df_test


def make_baseline(df_train, df_test):
    groupby_cols = ['store', 'item']
    store_item_table = (df_train.groupby(groupby_cols)
                                .agg({'sales': 'mean'})
                                .reset_index()
                                .rename(columns={'sales': 'baseline'})
                       )
    df_train = df_train.merge(store_item_table, left_on=groupby_cols, right_on=groupby_cols, how='left')
    df_test = df_test.merge(store_item_table, left_on=groupby_cols, right_on=groupby_cols, how='left')
    return df_train, df_test


def find_calendar_index(df, calendar):
    calendar_index = pd.pivot_table(df, index=calendar, values='sales', aggfunc=np.mean)
    calendar_index /= df['sales'].mean()
    return calendar_index.rename(columns={'sales': calendar+'_index'})


def make_calendar_adjustments(df_train, df_test):
    pred_col = 'baseline'
    for calendar in ['month', 'dayofweek', 'year']:
        if calendar in  ['month', 'dayofweek']:
            calendar_index = find_calendar_index(df_train, calendar)
        else:
            calendar_index = extrapolate_annual_growth(df_train)
        
        df_train = df_train.merge(calendar_index, left_on=calendar, right_on=calendar)
        df_train.loc[:, calendar+'_index'] = df_train.loc[:, calendar+'_index'].fillna(1.0)
        df_train = df_train.assign(pred=df_train[pred_col]*df_train[calendar+'_index'])\
                    .rename(columns={'pred': 'pred_'+calendar})
        
        df_test = df_test.merge(calendar_index, left_on=calendar, right_on=calendar)
        df_test.loc[:, calendar+'_index'] = df_test.loc[:, calendar+'_index'].fillna(1.0)
        df_test = df_test.assign(pred=df_test[pred_col]*df_test[calendar+'_index'])\
                    .rename(columns={'pred': 'pred_'+calendar})

        pred_col = 'pred_'+calendar

    return df_train, df_test


def extrapolate_annual_growth(df_train):
    fcst_horizons_year = 1
    calendar_index = find_calendar_index(df_train, 'year')
    # Find polynomial coefficients
    coeffs = np.polyfit(x=calendar_index.index.values.squeeze(), y=calendar_index.values.squeeze(), deg=2)
    p2 = np.poly1d(coeffs)
    years = np.arange(calendar_index.index.min(), calendar_index.index.max() + fcst_horizons_year + 1)
    calendar_index = pd.DataFrame({'year': years, calendar_index.columns.values[0]: p2(years)})
    return calendar_index

In [206]:
df_train_2, df_test_2 = naive_predictor_2(df_train, df_test)

In [207]:
output_csv(df_test_2)